<a href="https://colab.research.google.com/github/yayun502/AI-final-project/blob/main/baseline(rule-based).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
df_feel = pd.read_csv("/content/drive/MyDrive/final/feel.txt", delimiter=':', header=None, names=['sentence','label'])

In [3]:
df_feel

,sentence,label
0,'amused',"'joy',"
1,'beaming',"'joy',"
2,'blithe',"'joy',"
3,'buoyant',"'joy',"
4,'carefree',"'joy',"
...,...,...
324,'stupefied',"'surprise',"
325,'alarmed',"'surprise',"
326,'shock',"'surprise',"
327,'astonishment',"'surprise',"


In [4]:
for i in range(len(df_feel)):
  sentence = df_feel.loc[i, 'sentence']
  tmp = ''
  for c in sentence:
    if((c!='\'') & (c!=',')):
      tmp += c
  df_feel.loc[i, 'sentence'] = tmp
  label = df_feel.loc[i, 'label']
  tmp = ''
  for c in label:
    if((c!='\'') & (c!=',') & (c!=' ')):
      tmp += c
  df_feel.loc[i, 'label'] = tmp
df_feel

,sentence,label
0,amused,joy
1,beaming,joy
2,blithe,joy
3,buoyant,joy
4,carefree,joy
...,...,...
324,stupefied,surprise
325,alarmed,surprise
326,shock,surprise
327,astonishment,surprise


In [5]:
sentence_dict = dict()
for i in range(len(df_feel)):
  sentence_dict[df_feel.loc[i, 'sentence']] = df_feel.loc[i, 'label']
sentence_dict

{'abandoned': 'sadness',
 'accepted': 'love',
 'achy': 'sadness',
 'acrimony': 'anger',
 'admire': 'love',
 'adore': 'love',
 'affection': 'love',
 'affectionate': 'love',
 'agitated': 'sadness',
 'agony': 'sadness',
 'alarmed': 'surprise',
 'alone': 'sadness',
 'amused': 'joy',
 'anguish': 'sadness',
 'animosity': 'anger',
 'annoyance': 'anger',
 'annoyed': 'anger',
 'antisocial': 'sadness',
 'anxious': 'sadness',
 'astonished': 'surprise',
 'astonishment': 'surprise',
 'astounded': 'surprise',
 'awful': 'sadness',
 'baby': 'love',
 'beaming': 'joy',
 'beloved': 'love',
 'better': 'joy',
 'bewildered': 'surprise',
 'bitter': 'anger',
 'blessed': 'love',
 'blessing': 'love',
 'blithe': 'joy',
 'breakdown': 'sadness',
 'brittle': 'sadness',
 'broken': 'sadness',
 'brokenhearted': 'sadness',
 'buoyant': 'joy',
 'cared': '‘love’',
 'carefree': 'joy',
 'careworn': 'sadness',
 'catatonic': 'sadness',
 'cheerful': 'joy',
 'cheery': 'joy',
 'cherish': 'love',
 'chicken out': 'fear',
 'chipper

In [6]:
df_train = pd.read_csv("/content/drive/MyDrive/final/data/train.txt", delimiter=';', header=None, names=['sentence','label'])
df_test = pd.read_csv("/content/drive/MyDrive/final/data/test.txt", delimiter=';', header=None, names=['sentence','label'])
df_val = pd.read_csv("/content/drive/MyDrive/final/data/val.txt", delimiter=';', header=None, names=['sentence','label'])

In [7]:
df_train

,sentence,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
...,...,...
15995,i just had a very brief time in the beanbag an...,sadness
15996,i am now turning and i feel pathetic that i am...,sadness
15997,i feel strong and good overall,joy
15998,i feel like this was such a rude comment and i...,anger


In [8]:
df_total = pd.concat([df_train,df_test,df_val],axis=0, ignore_index = True)

In [9]:
def predict(sentence):
  count_dict = {'sadness': 0, 'anger': 0, 'love': 0, 'surprise': 0, 'fear': 0, 'joy': 0}
  sentence = sentence.split()
  # print('split = ', sentence)
  for word in sentence:
    if word in sentence_dict:
      if(sentence_dict[word] in count_dict):
        count_dict[sentence_dict[word]] += 1
  count_list = sorted(count_dict.items(), key=lambda x:x[1], reverse = True)
  # print('count_list = ', count_list)
  cls = [emo[0] for emo in count_list if emo[1]==count_list[0][1]]
  # print(count_list[0][1])
  # print('choosed classifications = ', cls)
  return random.choice(cls)

test = df_total.loc[15997, 'sentence']
print(test)
predict(test)


i feel strong and good overall


'joy'

In [10]:
def add_predict_col(df):
  df = df.reindex(columns = df.columns.tolist() + ['predict'])
  for row in range(len(df)):
    sen = df.loc[row, 'sentence']
    df.loc[row, 'predict'] = predict(sen)
  return df


In [11]:
df_total = add_predict_col(df_total)

In [12]:
train_data, valid_data = train_test_split(df_total,random_state=41,test_size=0.1)
train_data.reset_index(inplace=True, drop=True)
valid_data.reset_index(inplace=True, drop=True)

In [13]:
Emo = ['anger', 'fear', 'joy', 'love', 'sadness', 'surprise']
def micro_f(datas, predicts, labels, emotions):
  #datas: [str]
  #predicts: [predict]
  #labels: [label]
  #emotions: [emotion label]
  p = 0
  r = 0
  TP = 0
  FP = 0
  FN = 0
  n = 6
  for emotion in emotions:
    tp = 0
    fp = 0
    fn = 0
    for i in range(len(datas)):
      if predicts[i] == emotion:
        if predicts[i] == labels[i]:
          tp += 1
        else:
          fp += 1
      elif labels[i] == emotion:
        fn += 1
    TP += tp
    FP += fp
    FN += fn
  # print(TP)
  # print(FP)
  # print(FN)
  p = TP/(TP+FP)
  r = TP/(TP+FN)
  f = 2*p*r/(p+r)
  return (p, r, f)

def macro_f(datas, predicts, labels, emotions):
  #datas: [str]
  #predicts: [predict]
  #labels: [label]
  #emotions: [emotion label]
  p = 0
  r = 0
  n = 6
  for emotion in emotions:
    tp = 0
    fp = 0
    fn = 0
    for i in range(len(datas)):
      if predicts[i] == emotion:
        if predicts[i] == labels[i]:
          tp += 1
        else:
          fp += 1
      elif labels[i] == emotion:
        fn += 1
    p += tp/(tp+fp)
    r += tp/(tp+fn)
  p /= n
  r /= n
  f = 2*p*r/(p+r)
  return (p, r, f)


def accuracy(df):
  #evaluation
  correct = 0
  for i in range(len(df)):
    if df.loc[i, 'label'] == df.loc[i, 'predict']:
          correct += 1
  print('correct = ', correct, ' / ', len(df))
  eva = correct/len(df)
  print('accuracy = ', eva)


In [14]:
def compute_fscore(df):
  Microf = micro_f(df['sentence'].tolist(), df['predict'].tolist(), df['label'].tolist(), Emo)
  print('Microf: ', Microf)
  Macrof = macro_f(df['sentence'].tolist(), df['predict'].tolist(), df['label'].tolist(), Emo)
  print('Macrof: ', Macrof)
print('baseline performance')
print('for train')
compute_fscore(train_data)
accuracy(train_data)
print('for validation')
compute_fscore(valid_data)
accuracy(valid_data)

baseline performance
for train
Microf:  (0.32511111111111113, 0.32511111111111113, 0.32511111111111113)
Macrof:  (0.30832894302268793, 0.34015558286510084, 0.32346126126750746)
correct =  5852  /  18000
accuracy =  0.32511111111111113
for validation
Microf:  (0.3085, 0.3085, 0.3085)
Macrof:  (0.2888482309962842, 0.31715251779314274, 0.3023393746742682)
correct =  617  /  2000
accuracy =  0.3085
